# Introduccion
en este notebook se analizará, un dataset de 2000 canciones, y se espera crear una red neuronal capaz de poder el valor de popularidad de una canción

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [73]:
from keras import models
from keras import layers
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import category_encoders as ce
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

AttributeError: module 'tensorflow' has no attribute 'logging'

In [57]:
spotify = pd.read_csv('Spotify-2000.csv',
                      dtype={
                          'Top Genre':'category',
                          'Year':'category'},
                     decimal=",")
spotify.head()

,Index,Title,Artist,Top Genre,Year,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,1,Sunrise,Norah Jones,adult standards,2004,157,30,53,-14,11,68,201.0,94,3,71
1,2,Black Night,Deep Purple,album rock,2000,135,79,50,-11,17,81,207.0,17,7,39
2,3,Clint Eastwood,Gorillaz,alternative hip hop,2001,168,69,66,-9,7,52,341.0,2,17,69
3,4,The Pretender,Foo Fighters,alternative metal,2007,173,96,43,-4,3,37,269.0,0,4,76
4,5,Waitin' On A Sunny Day,Bruce Springsteen,classic rock,2002,106,82,58,-5,10,87,256.0,1,3,59


In [58]:
spotify.shape

(1994, 15)

#### Limpieza de datos

Eliminimo las columnas Index, Title (y Artist?) y que no son Features como tal 

In [59]:
spotify_clean = spotify.drop(columns=['Index', 'Title', 'Artist'])
spotify_clean.mean()

Beats Per Minute (BPM)    120.215647
Energy                     59.679539
Danceability               53.238215
Loudness (dB)              -9.008526
Liveness                   19.012036
Valence                    49.408726
Length (Duration)         259.842122
Acousticness               28.858074
Speechiness                 4.994985
Popularity                 59.526580
dtype: float64

Normalizamos las variables

In [60]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

columnas_numericas = spotify_clean.select_dtypes(include=numerics).columns.to_list()

In [61]:
maximo = spotify_clean['Popularity'].max()
minimo = spotify_clean['Popularity'].min()

In [62]:
spotify_clean[columnas_numericas] = (spotify_clean[columnas_numericas] - spotify_clean[columnas_numericas].min()) / (spotify_clean[columnas_numericas].max() - spotify_clean[columnas_numericas].min())
spotify_clean.head()

,Top Genre,Year,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,adult standards,2004,0.710059,0.278351,0.500000,0.52,0.092784,0.677083,0.207154,0.949495,0.018868,0.674157
1,album rock,2000,0.579882,0.783505,0.465116,0.64,0.154639,0.812500,0.213373,0.171717,0.094340,0.314607
2,alternative hip hop,2001,0.775148,0.680412,0.651163,0.72,0.051546,0.510417,0.352250,0.020202,0.283019,0.651685
3,alternative metal,2007,0.804734,0.958763,0.383721,0.92,0.010309,0.354167,0.277630,0.000000,0.037736,0.730337
4,classic rock,2002,0.408284,0.814433,0.558140,0.88,0.082474,0.875000,0.264156,0.010101,0.018868,0.539326


Ahora convierto la columna de Generos, de variable categorica (Top Genre y Year) a numerica, usando binary encoding

In [63]:
encoder = ce.BinaryEncoder(cols=['Top Genre', 'Year'],return_df=True)
spotify_clean = encoder.fit_transform(spotify_clean) 
spotify_clean.head()

/home/mauricio/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,Top Genre_0,Top Genre_1,Top Genre_2,Top Genre_3,Top Genre_4,Top Genre_5,Top Genre_6,Top Genre_7,Top Genre_8,Year_0,...,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,0,0,0,0,0,0,0,0,1,0,...,0.710059,0.278351,0.500000,0.52,0.092784,0.677083,0.207154,0.949495,0.018868,0.674157
1,0,0,0,0,0,0,0,1,0,0,...,0.579882,0.783505,0.465116,0.64,0.154639,0.812500,0.213373,0.171717,0.094340,0.314607
2,0,0,0,0,0,0,0,1,1,0,...,0.775148,0.680412,0.651163,0.72,0.051546,0.510417,0.352250,0.020202,0.283019,0.651685
3,0,0,0,0,0,0,1,0,0,0,...,0.804734,0.958763,0.383721,0.92,0.010309,0.354167,0.277630,0.000000,0.037736,0.730337
4,0,0,0,0,0,0,1,0,1,0,...,0.408284,0.814433,0.558140,0.88,0.082474,0.875000,0.264156,0.010101,0.018868,0.539326


Separo el dataset, por un lado 'X' va a tener de columnas todos los posibles features que se puede user, 'y' va a ser nuestra columna target, el valor que queremos predecir

In [64]:
X, y = spotify_clean.iloc[:,:-1], spotify_clean.iloc[:,-1]

In [65]:
n_features = X.columns.size

In [66]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [67]:
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)

In [76]:
def generadorRed(n_nuronas=32, n_capas_intermedias=4, lr=0.2):
    
    red = models.Sequential()

    red.add(layers.Dense(units=n_nuronas, activation='relu', input_shape=(n_features, )))

    for _ in range(n_capas_intermedias):
        red.add(layers.Dense(units=n_nuronas, activation='relu'))


    red.add(layers.Dense(units=1, activation='sigmoid'))

    # Compile neural network
    red.compile(loss='mean_absolute_error',  # Mean Error
                optimizer=SGD(lr=lr),  # Stocastic Gradiend Descent
                metrics=['mse'])  # Accuracy performance metric
    return red

In [87]:
red_neuronal = lambda n, ci, lr: KerasRegressor(build_fn=generadorRed,
                              epochs=2000,
                              batch_size=X_train.shape[0],
                              n_nuronas=n,
                              n_capas_intermedias=ci,
                              lr=lr,
                              verbose=0
                            )

In [88]:
## Crossvalidarion si es que queremos implementarlo mas adelante
# implemendanto gridSearch
learning_rate = [0.1, 0.2, 0.4, 0.7]
cantidad_neuronas = [4, 8, 16, 32]
cantidad_capas = [1, 2, 4, 8, 16]
scoreTotal = []
cantidad_folds = 5
cantidad_batch = 1
for lr in learning_rate:
    for neuronas in cantidad_neuronas:
        for capas in cantidad_capas:
            score = cross_val_score(red_neuronal(neuronas, capas, lr),
                                    X_train,
                                    y_train,
                                    cv=cantidad_folds,
                                    scoring='neg_mean_squared_error')

    # score = cross_val_score(red_neuronal, X_train, y_train, cv=5)
            print('----------------------------------')
            print('score:', score.mean())
            print('----------------------------------')
            scoreTotal.append(score.mean())

----------------------------------
score: -0.025564558058977126
----------------------------------
----------------------------------
score: -0.025782807543873788
----------------------------------


----------------------------------
score: -0.02640892453491688
----------------------------------
----------------------------------
score: -0.026578128337860107
----------------------------------


----------------------------------
score: -0.026759694144129754
----------------------------------
----------------------------------
score: -0.024569401517510414
----------------------------------


----------------------------------
score: -0.024524570256471635
----------------------------------
----------------------------------
score: -0.024591580778360368
----------------------------------


----------------------------------
score: -0.02602546475827694
----------------------------------
----------------------------------
score: -0.026813683658838273
----------------------------------


...

----------------------------------
score: -0.02345637045800686
----------------------------------
----------------------------------
score: -0.02416062019765377
----------------------------------


----------------------------------
score: -0.02546813152730465
----------------------------------
----------------------------------
score: -0.02415870539844036
----------------------------------


----------------------------------
score: -0.023993135988712312
----------------------------------
----------------------------------
score: -0.023881440237164497
----------------------------------


----------------------------------
score: -0.024137606099247932
----------------------------------
----------------------------------
score: -0.025342319160699844
----------------------------------
----------------------------------
score: -0.02482755109667778
----------------------------------


----------------------------------
score: -0.02512972243130207
----------------------------------


In [81]:
scoreTotal

[-0.12551778107881545,
 -0.130119164288044,
 -0.12527569830417634,
 -0.1306403622031212,
 -0.1327298402786255,
 -0.1210104450583458,
 -0.1257681131362915,
 -0.1260278344154358,
 -0.12997251003980637,
 -0.13071013987064362,
 -0.12022875100374222,
 -0.12214501202106476,
 -0.12098990529775619,
 -0.12042783796787263,
 -0.12978236377239227,
 -0.11787654906511306,
 -0.114938785135746,
 -0.12048978358507156,
 -0.1178139641880989,
 -0.1315624952316284,
 -0.12050658464431763,
 -0.12538360953330993,
 -0.12188553512096405,
 -0.1298457846045494,
 -0.13273718059062958,
 -0.11970551759004593,
 -0.12037627995014191,
 -0.11830671727657319,
 -0.1229331374168396,
 -0.1325469046831131,
 -0.11637531965970993,
 -0.11602785736322403,
 -0.12083341926336288,
 -0.12133269309997559,
 -0.12492233812808991,
 -0.11499754935503007,
 -0.1159154236316681,
 -0.12176190167665482,
 -0.12018437683582306,
 -0.12184653431177139,
 -0.12014817595481872,
 -0.12514200955629348,
 -0.12455497086048126,
 -0.13132281303405763,
 -0

In [ ]:
hist = red_neuronal.fit(X_train, y_train)

Los valores predecidos nos dan entre 0 y 1, asi que los desnormalizo

In [ ]:
y_pred = red_neuronal.predict(X_test)

In [ ]:
y_pred = y_pred * (maximo - minimo) + minimo


In [ ]:
y_lista = (y_test * (maximo - minimo) + minimo).to_list()


In [ ]:
np.mean((y_pred - y_lista)**2)

In [ ]:
y_test.size

In [ ]:
y_pred[4]

In [ ]:
y_lista[4]

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=[16,9])
t = np.arange(len(y_pred))
plt.scatter(t, y_pred)
plt.scatter(t, y_lista)


plt.show()

In [ ]:
plt.figure(figsize=[16,9])
plt.scatter(t, (y_pred - y_lista)**2)
plt.show()

In [ ]:
loss = hist.history['loss']
plt.figure(figsize=[16,9])
plt.plot(np.arange(len(loss)), loss)
plt.show()